# **Spark Apache (Семинары)**
## **Урок 2. Инструменты работы и визуализации ч.1**
### Задание:
1. Условие: Дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров

Данные для таблицы:

In [1]:
# Данные для задания
data = {'id':[1, 2, 3, 4], 'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary': ['70', '80', '60', '90'], 'ManagerId': ['3', '4', 'Null', 'Null']}

In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1b5d6bda3c2a4735ea54fc269fc0d37b62d3045694fc73c3b41d3ec6ab677281
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
# Установка требуемых модулей
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd

In [4]:
# Установка сессии spark
spark = SparkSession.builder.appName('AggregateTable').getOrCreate()

# Создаем таблицу
df_spark = spark.createDataFrame(pd.DataFrame(data))

# Конвертируем данные (string -> int)
df_spark = df_spark.withColumn('Salary',df_spark['Salary'].cast('int'))
df_spark = df_spark.withColumn('ManagerId',df_spark['ManagerId'].cast('int'))

# Смотрим результат
df_spark.show()

+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     NULL|
|  4|  Max|    90|     NULL|
+---+-----+------+---------+



In [15]:
# Обработка данных по заданию
df_result = df_spark.alias('a').join(df_spark.alias('b')['id', 'Salary'].withColumnRenamed("Salary", "Salary_b"), \
                                     col('a.ManagerId') == col('b.id'), \
                                     'inner') \
                               .filter('Salary > Salary_b') \
                               .select(['a.Name'])
df_result.show()

+----+
|Name|
+----+
| Joe|
+----+



In [8]:
spark.stop()



---

---

---

#**Пояснения к проделанной работе**

Изначальная таблица данных:

In [9]:
# Установка сессии spark
spark = SparkSession.builder.appName('AggregateTable').getOrCreate()
# Создаем таблицу
df_spark = spark.createDataFrame(pd.DataFrame(data))
# Конвертируем данные (string -> int)
df_spark = df_spark.withColumn('Salary',df_spark['Salary'].cast('int'))
df_spark = df_spark.withColumn('ManagerId',df_spark['ManagerId'].cast('int'))
# Смотрим результат
df_spark.show()

+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     NULL|
|  4|  Max|    90|     NULL|
+---+-----+------+---------+



In [12]:
# Объединяем таблицу саму с собой по полям id = ManagerId. Тип объединения inner
df_result = df_spark.alias('a').join(df_spark.alias('b')['id', 'Salary'].withColumnRenamed("Salary", "Salary_b"), \
                                     col('a.ManagerId') == col('b.id'), \
                                     'inner')
df_result.show()

+---+-----+------+---------+---+--------+
| id| Name|Salary|ManagerId| id|Salary_b|
+---+-----+------+---------+---+--------+
|  1|  Joe|    70|        3|  3|      60|
|  2|Henry|    80|        4|  4|      90|
+---+-----+------+---------+---+--------+



Строки с ID равными 3 и 4 не вошли в результирующую выборку. Осталось 2 строки.

Далее (по условию задачи) - смотрим на строки, в которых Salary > Salary_b

In [14]:
df_result = df_spark.alias('a').join(df_spark.alias('b')['id', 'Salary'].withColumnRenamed("Salary", "Salary_b"), \
                                     col('a.ManagerId') == col('b.id'), \
                                     'inner') \
                               .filter('Salary > Salary_b')
df_result.show()

+---+----+------+---------+---+--------+
| id|Name|Salary|ManagerId| id|Salary_b|
+---+----+------+---------+---+--------+
|  1| Joe|    70|        3|  3|      60|
+---+----+------+---------+---+--------+



Остается одна строка с требуемой записью. Из нее (по условию задачи) выбираем поле Name

In [16]:
df_result = df_spark.alias('a').join(df_spark.alias('b')['id', 'Salary'].withColumnRenamed("Salary", "Salary_b"), \
                                     col('a.ManagerId') == col('b.id'), \
                                     'inner') \
                               .filter('Salary > Salary_b') \
                               .select(['a.Name'])
df_result.show()

+----+
|Name|
+----+
| Joe|
+----+



Получаем требуемый результат.

In [18]:
spark.stop()